# Simple backpropagation in Devito

In this example, we'll implement a simple convolutional neural network (CNN) in Devito, run a forward pass through it and then use backpropagation to obtain gradients necessary for training.

The CNN will have the following structure:
1. Max pool layer: input size 1x2x4x4, kernel size 2x2, stride 1x1
2. Convolutional layer: input size 1x2x3x3, kernel size 2x2x2, stride 1x1, activation ReLU
3. Flattening layer
4. Fully connected layer: input size 8x1, kernel size 3x8, activation softmax

*Size glossary: batch size x channels x height x width **or** output channels x height x width **or** height x width. Height and width are equivalent to rows and columns.*

All parameters for the forward pass will be (pseudo)random numbers generated by `np.random.rand`. Therefore, different results will be obtained each time the notebook is executed.

In [1]:
import joey
import numpy as np
from sympy.functions import sign
from devito import Operator, Eq, Inc
from joey.activation import ReLU

In [2]:
def backward_pass(conv_kernel, conv_bias, fc_weights, fc_bias, input_data, expected_results):
    layer1 = joey.MaxPooling(kernel_size=(2, 2),
                             input_size=(1, 2, 4, 4),
                             generate_code=False)
    layer2 = joey.Conv(kernel_size=(2, 2, 2),
                       input_size=(1, 2, 3, 3),
                       activation=ReLU(),
                       generate_code=False)
    layer_flat = joey.Flat(input_size=(1, 2, 2, 2),
                           generate_code=False)
    layer3 = joey.FullyConnectedSoftmax(weight_size=(3, 8),
                                        input_size=(8, 1),
                                        generate_code=False)
    
    eqs = layer1.equations() + layer2.equations(layer1.result) + \
            layer_flat.equations(layer2.result) + layer3.equations(layer_flat.result)
    
    op = Operator(eqs)
    
    layer2.kernel.data[:] = conv_kernel
    layer2.bias.data[:] = conv_bias
    
    layer3.kernel.data[:] = fc_weights
    layer3.bias.data[:] = fc_bias
    
    layer1.input.data[:] = input_data
    
    op.apply()
    
    gradients = []
    
    for i in range(3):
        result = layer3.result.data[i]
        if i == expected_results[0]:
            result -= 1
        gradients.append(result)
    
    layer3.result_gradients.data[:] = gradients
    
    dims = [layer3.bias_gradients.dimensions[0],
            layer3.kernel_gradients.dimensions[0],
            layer3.kernel_gradients.dimensions[1],
            layer_flat.result_gradients.dimensions[0],
            layer3.kernel.dimensions[1],
            layer3.result_gradients.dimensions[0],
            layer2.result_gradients.dimensions[0],
            layer2.result_gradients.dimensions[1],
            layer2.result_gradients.dimensions[2],
            layer2.kernel_gradients.dimensions[0],
            layer2.kernel_gradients.dimensions[1],
            layer2.kernel_gradients.dimensions[2],
            layer2.kernel_gradients.dimensions[3],
            layer2.bias_gradients.dimensions[0]]
    
    _, _, layer2_height, layer2_width = layer2.kernel.shape
    
    backprop_eqs = [
        Eq(layer3.bias_gradients[dims[0]], layer3.result_gradients[dims[0]]),
        Eq(layer3.kernel_gradients[dims[1], dims[2]],
           layer_flat.result[dims[2], 0] * layer3.result_gradients[dims[1]]),
        Inc(layer_flat.result_gradients[dims[4]],
            layer3.kernel[dims[5], dims[4]] * layer3.result_gradients[dims[5]]),
        Eq(layer_flat.result_gradients[dims[3]],
           layer_flat.result_gradients[dims[3]] * sign(layer_flat.result[dims[3], 0])),
        Eq(layer2.result_gradients[dims[6], dims[7], dims[8]],
           layer_flat.result_gradients[dims[6] * layer2_height * layer2_width + dims[7] * layer2_height + dims[8]]),
        Inc(layer2.bias_gradients[dims[13]], layer2.result_gradients[dims[13], dims[7], dims[8]]),
        Eq(layer2.kernel_gradients[dims[9], dims[10], dims[11], dims[12]],
            sum([layer2.result_gradients[dims[9], x, y] * layer1.result[0, dims[10], dims[11] + x, dims[12] + y]
                 for x in range(layer2.result_gradients.shape[1])
                 for y in range(layer2.result_gradients.shape[2])]))
    ]
    
    backprop_op = Operator(backprop_eqs)
    backprop_op.apply()
    
    return (layer2.kernel_gradients.data, layer2.bias_gradients.data,
            layer3.kernel_gradients.data, layer3.bias_gradients.data)

In [3]:
conv_kernel = np.random.rand(2, 2, 2)
conv_bias = np.random.rand(2)

fc_weights = np.random.rand(3, 8)
fc_bias = np.random.rand(3)

In [4]:
input_data = np.array([[[[1, 2, 3, 4],
                         [5, 6, 7, 8],
                         [9, 10, 11, 12],
                         [13, 14, 15, 16]],
                        [[-1, -2, 0, 1],
                         [-2, -3, 1, 2],
                         [3, 4, 2, -1],
                         [-2, -3, -4, 9]]]],
                     dtype=np.float64)
expected = np.array([2])

In [5]:
conv_kernel_grad, conv_bias_grad, fc_kernel_grad, fc_bias_grad = backward_pass(conv_kernel, conv_bias, fc_weights, fc_bias, input_data, expected)

/home/maksymilian/Desktop/UROP/devito/devito/types/grid.py:206: RuntimeWarning: divide by zero encountered in true_divide
  spacing = (np.array(self.extent) / (np.array(self.shape) - 1)).astype(self.dtype)
Operator `Kernel` run in 0.01 s
Operator `Kernel` run in 0.01 s


Here are kernel gradients of the convolutional layer:

In [6]:
print(conv_kernel_grad)

[[[[ 1.77081810e-10  2.19350411e-10]
   [ 3.46156214e-10  3.88424815e-10]]

  [[-1.00338299e-10  1.88046451e-11]
   [ 1.69074404e-10  5.35400815e-11]]]


 [[[-6.81563531e-10 -7.66421514e-10]
   [-1.02099546e-09 -1.10585345e-09]]

  [[-1.29241071e-10 -1.51084775e-10]
   [-3.39431933e-10 -3.87942350e-10]]]]


Here are bias gradients of the convolutional layer:

In [7]:
print(conv_bias_grad)

[ 4.22686009e-11 -8.48579832e-11]


Here are kernel gradients of the fully connected layer:

In [8]:
print(fc_kernel_grad)

[[ 2.36506389e-14  2.71186908e-14  3.67867182e-14  3.78706765e-14
   2.36562622e-14  2.71243141e-14  3.67923415e-14  3.78762998e-14]
 [ 8.17835348e-10  9.37760035e-10  1.27207889e-09  1.30956200e-09
   8.18029802e-10  9.37954489e-10  1.27227334e-09  1.30975645e-09]
 [-8.17857696e-10 -9.37785659e-10 -1.27211365e-09 -1.30959778e-09
  -8.18052155e-10 -9.37980118e-10 -1.27230811e-09 -1.30979224e-09]]


Here are bias gradients of the fully connected layer:

In [9]:
print(fc_bias_grad)

[ 1.45661929e-15  5.03696643e-11 -5.03710407e-11]


## Comparison with PyTorch
To check correctness, we will carry out the same activities using PyTorch.

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Conv2d(2, 2, 2)
        self.fc = nn.Linear(8, 3)
        
    def forward(self, x):
        x = F.max_pool2d(x, 2, stride=(1, 1))
        x = F.relu(self.conv(x))
        x = x.view(-1, self.num_flat_features(x))
        x = self.fc(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [12]:
net = Net()
net.double()

with torch.no_grad():
    net.conv.weight[:] = torch.from_numpy(conv_kernel)
    net.conv.bias[:] = torch.from_numpy(conv_bias)
    
    net.fc.weight[:] = torch.from_numpy(fc_weights)
    net.fc.bias[:] = torch.from_numpy(fc_bias)

In [13]:
criterion = nn.CrossEntropyLoss()
input_data_tensor = torch.from_numpy(input_data)
outputs = net(input_data_tensor)
net.zero_grad()
loss = criterion(outputs, torch.from_numpy(expected))
loss.backward()

Here is the relative convolutional layer kernel error along with the maximum:

In [14]:
pytorch_conv_kernel_grad = net.conv.weight.grad.numpy()
conv_kernel_error = abs(conv_kernel_grad - pytorch_conv_kernel_grad) / abs(pytorch_conv_kernel_grad)
print(conv_kernel_error)
print(np.amax(conv_kernel_error))

[[[[6.62723345e-14 6.04545902e-14]
   [5.15261066e-14 4.97788661e-14]]

  [[1.72607014e-14 1.41930354e-13]
   [3.59286059e-14 1.23115156e-13]]]


 [[[9.40579925e-15 9.30875850e-15]
   [9.51950576e-15 9.53702576e-15]]

  [[8.00036521e-15 1.04366111e-14]
   [9.59551341e-15 1.01280716e-14]]]]
1.419303541865712e-13


Here is the relative convolutional layer bias error along with the maximum:

In [15]:
pytorch_conv_bias_grad = net.conv.bias.grad.numpy()
conv_bias_error = abs(conv_bias_grad - pytorch_conv_bias_grad) / abs(pytorch_conv_bias_grad)
print(conv_bias_error)
print(np.amax(conv_bias_error))

[3.59286059e-14 9.44320367e-15]
3.592860592048525e-14


Here is the relative fully connected layer kernel error along with the maximum:

In [16]:
pytorch_fc_kernel_grad = net.fc.weight.grad.numpy()
fc_kernel_error = abs(fc_kernel_grad - pytorch_fc_kernel_grad) / abs(pytorch_fc_kernel_grad)
print(fc_kernel_error)
print(np.amax(fc_kernel_error))

[[1.36087338e-14 1.34973868e-14 1.38958269e-14 1.36647349e-14
  1.37388861e-14 1.34945886e-14 1.37221758e-14 1.36627061e-14]
 [1.37806905e-14 1.34517402e-14 1.38180015e-14 1.37383174e-14
  1.37774147e-14 1.36694260e-14 1.39784294e-14 1.37362777e-14]
 [0.00000000e+00 2.20514305e-16 1.62560281e-16 0.00000000e+00
  0.00000000e+00 2.20468589e-16 1.62535436e-16 0.00000000e+00]]
1.3978429432582165e-14


Here is the relative fully connected layer bias error along with the maximum:

In [17]:
pytorch_fc_bias_grad = net.fc.bias.grad.numpy()
fc_bias_error = abs(fc_bias_grad - pytorch_fc_bias_grad) / abs(pytorch_fc_bias_grad)
print(fc_bias_error)
print(np.amax(fc_bias_error))

[1.36746355e-14 1.37279313e-14 0.00000000e+00]
1.3727931341179665e-14


Here is the maximum overall error:

In [18]:
print(max(np.amax(conv_kernel_error), np.amax(conv_bias_error), np.amax(fc_kernel_error), np.amax(fc_bias_error)))

1.419303541865712e-13
